#**ECE 4554/ ECE 5554 / Computer Vision**
This file contains Machine Problems 1 and 2 for Homework 4. Your job is to implement/modify the sections within this notebook that are marked with "TO DO".

##**TO DO**: Enter your Virginia Tech Username (PID) here: ____________________ 

##**Honor Code**

Please review the Honor Code statement in the syllabus.  This is not a "team project".  *Also, if you make use of code from ANY source except the instructor, you must provide comment lines in your programs to state where you obtained that code.* Failure to cite sources of code that you borrowed will be considered a violation of the Honor Code.

##**Code libraries**
You are allowed to use NumPy and Matplotlib functions to perform matrix operations and graphics/plotting operations. You are  allowed to use OpenCV functions for graphics/plotting, such as `cv2.circle` and `cv2.line`. You are also allowed to use any OpenCV functions that are provided in this start-up notebook, but do not use any other OpenCV functions without permission from the instructor.

##**Submission guidelines** for the coding problems (Google Colab)

1. Please verify that you have entered your Virginia Tech Username in all of the appropriate places.
2. After clicking Runtime->Run all, verify that all of your solutions are visible in this notebook.
3. Click File->Save near the top of the page to save the latest version of your notebook at Google Drive.
4. Verify that the last 2 cells have executed, creating a PDF version of this notebook at Google Drive. (Note: if you face difficulty with this step, please refer to https://pypi.org/project/notebook-as-pdf/)
5. Look at the PDF file and check that all of your solutions are displayed correctly there.
6. Download your notebook file and the PDF version to your laptop.
7. On your laptop, create a ZIP version of this notebook file. (Please don't include the separate data files.) Use file name Homework4_Code_USERNAME.zip, with your own Username.
6. For your PDF version, use file name Homework4_Notebook_USERNAME.pdf, with your own Username.
7. **Submit these 2 files and your PDF file for Problems 1-4 SEPARATELY to Canvas.** Do not zip them all together.

##**Overview**

For Machine Problem 1, you will write code to find the fundamental matrix for a stereo image pair. You will also write code to display keypoints and epipolar lines in the two images. For this problem, you may assume that no outliers are present in your set of corresponding image points.

In Machine Problem 2, you will extend the previous problem by using RANSAC to address the problem of outliers. 





# Environment setup

In [ ]:
# Mount your Google Drive to this notebook
# The purpose is to allow your code to access to your files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change the directory to your own working directory
# Any files under your working directory are available to your code
# TO DO: enter the name of your directory
import os
os.chdir('/content/drive/MyDrive/5554/HW4')


In [ ]:
# Import library modules
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import optimize 
from PIL import Image  # Python Imaging Library
# The following is a substitute for cv2.imshow, 
#  which you would use on your local machine but Colab does not support it
from google.colab.patches import cv2_imshow

print('Python version:', sys.version)
print('OpenCV version:', cv2.__version__)
print('NumPy version: ', np.__version__)


#Getting started

Several image files were provided to you (Durham*.png and others). Upload them to your working directory.

The following functions are helpful for loading images into floating-point format, and for displaying images that are in that format.  Let's use those functions to display a stereo image pair. 


In [ ]:
def load_image(filename):
  img = np.asarray(Image.open(filename))
  img = img.astype("float32")/255.0
  return img

def show_image(img):
  fig = plt.figure()
  fig.set_size_inches(16, 10) # You can adjust the size of the displayed figure
  plt.imshow(img)

left_img = load_image("Durham1.png")
right_img = load_image("Durham2.png")
show_image(np.concatenate([left_img, right_img], axis=1))

Now verify that you can use OpenCV tools to detect SIFT-based keypoints. The following code block should detect keypoints in one of the Durham images, and then display the image using small small circles to indicate keypoint locations.

These library functions allow for many options. For example, if you change the `nfeatures` parameter for `SIFT_create`, you'll see different numbers of detected keypoints. You may want to experiment with these parameters later.

In [ ]:
def testSIFT(img1):
  sift = cv2.SIFT_create(nfeatures=500)
  kp = sift.detect(img1, None)
  img1=cv2.drawKeypoints(img1, kp, None, color=(0, 0, 255))
  
  fig = plt.figure()
  fig.set_size_inches(12, 9) # You can adjust the size of the displayed figure
  plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))


img = cv2.imread("Durham1.png", cv2.IMREAD_COLOR)
testSIFT(img)

Next, verify that you can use a matching technique from OpenCV that tries to detect corresponding keypoints between 2 images. This example displays an example with a small number of correspondences.

The library call cv2.BFmatcher() is OpenCV's "brute force" matcher. More description is given in 
https://github.com/abidrahmank/OpenCV2-Python-Tutorials/blob/master/source/py_tutorials/py_feature2d/py_matcher/py_matcher.rst

In [ ]:
def genSIFTMatchPairs(img1, img2, numberOfMatches):
  # Find keypoints and their SIFT descriptors in the two images
  sift = cv2.SIFT_create()
  kp1, des1 = sift.detectAndCompute(img1, None)
  kp2, des2 = sift.detectAndCompute(img2, None)

  # Search for corresponding pairs of points 
  bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
  matches = bf.match(des1, des2)

  # Sort them based on distance (dissimilarity) between two descriptors
  matches = sorted(matches, key = lambda x:x.distance)

  pts1 = np.zeros((numberOfMatches, 2))
  pts2 = np.zeros((numberOfMatches, 2))
  for i in range(numberOfMatches):
    pts1[i,:] = kp1[matches[i].queryIdx].pt
    pts2[i,:] = kp2[matches[i].trainIdx].pt
  return pts1, pts2, matches[:numberOfMatches], kp1, kp2

def testMatchingProcedure():
  img1 = cv2.imread('Durham1.png')
  img2 = cv2.imread('Durham2.png')
  
  # Perform the matching step (here, only the best 11 correspondences are returned)
  pts1, pts2, matches1to2, kp1, kp2 = genSIFTMatchPairs(img1, img2, 11)
  # Now pts1 and pts2 should contain lists of corresponding points 
  #  within img1 and img2, respectively
  #  Each point has the format [column row] 
  
  # Display the two images to check the results visually
  #  (the parameter flags=2 removes unmatched points from the display) 
  matching_result = cv2.drawMatches(img1, kp1, img2, kp2, matches1to2, None, flags=2,
    matchColor = (255, 255, 0), singlePointColor=(0, 0, 255))
  fig = plt.figure()
  fig.set_size_inches(16, 10) # You can adjust the size of the displayed figure
  plt.imshow(cv2.cvtColor(matching_result, cv2.COLOR_BGR2RGB))


testMatchingProcedure()

#Machine Problem 1: Fundamental matrix

The previous steps have shown how you can use OpenCV to find several pairs of corresponding points in two images. For this problem, you need to detect correspondences and use them to solve for the fundamental matrix *F* that relates the two images. You must demonstrate a correct result by using *F* to draw epipolar lines for keypoints that you detect.

The main parts of this problem are as follows:

*   **Part (a)** Write code that implements the **8-point algorithm to compute *F*.** For a given set of 8 or more correspondences, find a least-squares solution based on  those image points. An outline of the approach is given in the lecture slides: packet 17, page 23. 
*   **Part (b)** Write a function that uses *F* to **compute the epipoles** for the two images.
*   **Part (c)** Write a function to **display two images and highlight keypoints** that have been detected in both images. Also **display epipolar lines** that you obtain using your computed *F* matrix. One or both images should show epipolar lines passing through (or near) those keypoints. 
*   **Part (d)** Write a function that **combines all of the steps**, including keypoint detection and matching.

You do not need to consider outliers for this Machine Problem, and you do not need to consider additional steps that were described in the "Gold standard" algorithm from the lecture slides.

**Part 1(a).**
Implement the following function, as described above.



In [ ]:
##############################
# TO DO: Implement the findFundamentalMatrix function
#  (You are allowed to subdivide your code into additional functions that are called by this one)
#
def findFundamentalMatrix(pts1, pts2, scale):
  '''Use the eight-point algorithm to compute the fundamental matrix for the given points. 
   Input:
    pts1: keypoint locations for image 1, shape (N, 2), where N >= 8
    pts2: keypoint locations for image 2, with a corresponding point for every one in pts1 
    scale: use this scale factor to convert all coordinates to the range [0, 1]
           (see the additional notes that have been provided for this problem)
   Returns:
    F: the fundamental matrix, shape (3, 3)  
  '''
  # The following line is just a placeholder. Replace it with your code.
  F = np.asarray([[0, 0, 0],[0, 0, -1],[0, 1, 0]])
  

  return F



**Part 1(b).** Write a function that uses *F* to compute the epipoles for the 
two images.

In [ ]:
##############################
# TO DO: Implement the findEpipoles function
#
def findEpipoles(F):
  '''Compute the two epipoles from a given fundamental matrix
   Input:
    F: fundamental (or essential) matrix, shape (3, 3)
   Returns:
    e1, e2: the two epipoles; each is a vector with 3 components
  '''
  # The following 2 lines are just placeholders. Replace them with your code.
  e1 = np.asarray([1, 0, 1])
  e2 = np.asarray([0, 1, 1])



  return e1, e2
  

For grading purposes, run the following code block to print your computed matrix *F* and the two epipoles for the sample points that are given below.

In [ ]:
##############################
# Do not modify this code block 
#  (For debugging, you could copy this code to another block and edit it there)

# The following are some matching keypoints that were detected in the Durham1 and Durham2 images.
pts1_list = [
  [224.95256042, 321.64755249],
  [280.72879028, 296.15835571],
  [302.34194946, 364.82437134],
  [434.68283081, 402.86990356],
  [244.64321899, 308.50286865],
  [488.62979126, 216.26953125],
  [214.77470398, 430.75869751],
  [299.20846558, 312.07217407],
  [266.94125366, 119.36679077],
  [384.41549683, 442.05865479],
  [475.28448486, 254.28138733]]

pts2_list = [
  [253.88285828, 335.00772095],
  [304.884552,   308.89205933],
  [325.33914185, 375.91308594],
  [455.15515137, 411.18075562],
  [271.48794556, 322.07028198],
  [515.11816406, 221.74610901],
  [245.31390381, 441.54830933],
  [321.74771118, 324.31417847],
  [289.86627197, 137.46456909],
  [403.3711853,  451.08905029],
  [496.16610718, 261.36074829]]
 
pts1 = np.asarray(pts1_list)  
pts2 = np.asarray(pts2_list)

# Compute F and the epipoles
#  Scale factor 800 is the largest (row, column) size of the Durham images
F = findFundamentalMatrix(pts1, pts2, 800)
e1, e2 = findEpipoles(F)

# Print F and the epipoles
print('F =\n', F)
print('\n')
print('normalized epipole #1:', e1/e1[2])
print('normalized epipole #2:', e2/e2[2])  


**Part 1(c).** 
Write a function `displayEpipolarLines` that will display two stereo images and highlight keypoints in both images. Also display epipolar lines in at least one of the images. This function should display information for all of the keypoints that are provided to it. For practical reasons, usually it is best to display a small number of keypoints (e.g., 15 or 20, not 200) so that it is relatively easy to see the individual keypoints and lines. 



In [ ]:
##############################
# TO DO: Implement the displayEpipolarLines function
#
def displayEpipolarLines(pts1, pts2, F, img1, img2):
  '''Display two images with highlighted keypoints and epipolar lines. 
     Assume that the keypoints have already been detected, and F has already been computed.
   Input:
    pts1: keypoint locations for image 1, shape (N, 2), where N >= 1
    pts2: keypoint locations for image 2, with a corresponding point for every one in pts1
    F:  fundamental matrix for the two images 
    img1: image 1
    img2: image 2
   Output:
    No return value; the output is a visual display  
  '''
  # write your code here
  


Run the following code block to test your code. This test uses the precomputed set of 11 correspondences from the Durham images. (Here I'm using 11 correspondences because my least-squared results were much worse with a smaller number. Possible reasons: not well distributed spatially across the images, or lens distortion, etc.) 

In [ ]:
##############################
# Do not modify this code block 
#  (For debugging, you could copy this code to another block and edit it there)
#
def testEpipolarLines():
  img1 = cv2.imread('Durham1.png')
  img2 = cv2.imread('Durham2.png')
  
  # The following are some matching keypoints that were detected in the Durham1 and Durham2 images.
  pts1_list = [
    [224.95256042, 321.64755249],
    [280.72879028, 296.15835571],
    [302.34194946, 364.82437134],
    [434.68283081, 402.86990356],
    [244.64321899, 308.50286865],
    [488.62979126, 216.26953125],
    [214.77470398, 430.75869751],
    [299.20846558, 312.07217407],
    [266.94125366, 119.36679077],
    [384.41549683, 442.05865479],
    [475.28448486, 254.28138733]]

  pts2_list = [
    [253.88285828, 335.00772095],
    [304.884552,   308.89205933],
    [325.33914185, 375.91308594],
    [455.15515137, 411.18075562],
    [271.48794556, 322.07028198],
    [515.11816406, 221.74610901],
    [245.31390381, 441.54830933],
    [321.74771118, 324.31417847],
    [289.86627197, 137.46456909],
    [403.3711853,  451.08905029],
    [496.16610718, 261.36074829]]

  pts1 = np.asarray(pts1_list)  
  pts2 = np.asarray(pts2_list)

  # Compute F and the epipoles
  #  Scale factor 800 is the largest (row, column) size of the Durham images
  Fmat = findFundamentalMatrix(pts1, pts2, 800)
  e1tmp, e2tmp = findEpipoles(Fmat)

  # Print F and the epipoles
  print('Fmat =\n', Fmat)
  print('\n')
  print('normalized epipole #1:', e1tmp/e1tmp[2])
  print('normalized epipole #2:', e2tmp/e2tmp[2]) 
  
  # Display the two images with epipolar lines highlighted
  displayEpipolarLines(pts1, pts2, Fmat, img1, img2)


testEpipolarLines()


**Part 1(d).**
Write a function that performs the full sequence of operations: load two images, detect SIFT keypoints, perform stereo matching, compute the fundamental matrix, and display the results.

Show your results using the files `operahouse1.png` and `operahouse2.png`.
For all steps up to performing stereo matching, you may borrow liberally from the "starter code" in this notebook, although you may need to adjust some of the parameters in order to get good results. To compute the fundamental matrix, you must call your function `findFundamentalMatrix`. To display the results, you must call your function `displayEpipolarLines`.


In [ ]:
##############################
# TO DO: Implement the following function 
#
def stereoMatchingAndDisplay(img1file, img2file):
  '''Extract keypoints from a stereo image pair, find corresponding points, 
      compute the fundamental matrix (assuming no outliers), 
      and display the results with sample epipolar lines. 
   Input:
    img1file: name of file containing image 1
    img2file: name of file containing image 2
   Output:
    No return value; the output is a visual display  
  '''
  # write your code here






Run the following code block to test your  `stereoMatchingAndDisplay` function.

In [ ]:
##############################
# Please do not change the first line of code in this block, for the operahouse pair.
#  It is okay (but not required) to add more lines that test your stereoMatchingAndDisplay 
#  function with additional stereo pairs.
#
stereoMatchingAndDisplay('operahouse1.png', 'operahouse2.png') 



# Machine Problem 2: Using RANSAC to deal with outliers

In the previous problem, undoubtedly you saw that incorrect correspondences can be reported by the OpenCV tools. It is very likely that those false matches will act as outliers in your estimation of the fundamental matrix. 

As discussed during lectures, the RANSAC algorithm is a popular way to deal with outliers during model fitting. Here you must implement the RANSAC approach to compute the fundamental matrix *F*. 

A suggestion is that you detect *N*>20 correspondences from an image pair, and repeatly call your `findFundamentalMatrix` function with only 8 correspondences that are drawn at random. Based on the computed *F*, determine which correspondences are inliers, and refine your estimate of *F* using all of the inliers.

*Note that this problem is more open-ended than Machine Problem 1.* You need to make choices about what parameters to use, what input images to process, and what results to report. Use comments and print statements (as described below) to tell the grader what you have done. Try to make your explanations very clear to the grader.


In [ ]:
##############################
# TO DO: Implement the following function
#  (You are allowed to subdivide your code into additional functions that are called by this one)
#
def findFundamentalMatrixUsingRansac(pts1, pts2, scale):
  '''Use the eight-point algorithm  to compute the fundamental matrix for the given points,
     using RANSAC to deal with outliers. 
   Input:
    pts1: keypoint locations for image 1, shape (N, 2), where N >= 8
    pts2: keypoint locations for image 2, with a corresponding point for every one in pts1 
    scale: use this scale factor to convert all coordinates to the range [0, 1]
   Returns:
    F: the fundamental matrix, shape (3, 3)  
  '''
  # The following line is just a placeholder. Replace it with your code.
  F = np.asarray([[0, 0, 0],[0, 0, -1],[0, 1, 0]])

  return F



In the following code block, write code to test the RANSAC method that you have implemented in `findFundamentalMatrixUsingRansac`. Try to select an image pair and algorithm parameters for which RANSAC gives a good improvement over the non-RANSAC 8-point algorithm. Generate and display outputs for both cases, as a  comparison of your results *with* RANSAC and your results *without* RANSAC. Use print statements liberally to explain to the grader the different outputs that you have generated.  As part of your code's comments, describe how you decide which correspondences are outliers. (If convenient, you may also provide additional descriptions in a text block below.)

In [ ]:
##############################
# TO DO: Write code to test your RANSAC implementation (your function findFundamentalMatrixUsingRansac)
# 






---
# Creating a PDF version of your current notebook

In [ ]:
#The following two installation steps are needed to generate a PDF version of the notebook
#(These lines are needed within Google Colab, but are not needed within a local version of Jupyter notebook)
!apt-get -qq install texlive texlive-xetex texlive-latex-extra pandoc 
!pip install --quiet pypandoc

In [ ]:
# TO DO: Provide the full path to your Jupyter notebook file
!jupyter nbconvert --to PDF "/content/drive/MyDrive/5554/HW4/Homework4_USERNAME.ipynb"